## Part 1: Decision trees (DSFS Chapter 17)

> *Exercise: Building the minority report algorithm*
>* *Use the category of the crimes (e.g. ASSAULT, LARCENY/THEFT, ...) to build a decision tree that predicts the corresponding district (e.g. TENDERLOIN, MISSION, etc). I recommend useing the DecisionTreeClassifier class in scikit-learn (but you can also try the ID3 tree in the DSFS book - note, though, that this one only does binary classification). For training, use 90% of the data and test the tree prediction on the remaining 10%.*

In [22]:
import pandas as pd
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [6]:
# Load it into a Dataframe using pandas
path = '../data/sfpd_incidents.csv'
df = pd.read_csv(path)
df.head()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,150060275,NON-CRIMINAL,LOST PROPERTY,Monday,01/19/2015,14:00,MISSION,NONE,18TH ST / VALENCIA ST,-122.421582,37.761701,"(37.7617007179518, -122.42158168137)",15006027571000
1,150098210,ROBBERY,"ROBBERY, BODILY FORCE",Sunday,02/01/2015,15:45,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191,"(37.7841907151119, -122.414406029855)",15009821003074
2,150098210,ASSAULT,AGGRAVATED ASSAULT WITH BODILY FORCE,Sunday,02/01/2015,15:45,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191,"(37.7841907151119, -122.414406029855)",15009821004014
3,150098210,SECONDARY CODES,DOMESTIC VIOLENCE,Sunday,02/01/2015,15:45,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191,"(37.7841907151119, -122.414406029855)",15009821015200
4,150098226,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Tuesday,01/27/2015,19:00,NORTHERN,NONE,LOMBARD ST / LAGUNA ST,-122.431119,37.800469,"(37.8004687042875, -122.431118543788)",15009822628160


In [26]:
# Keep only data used for decision tree
df_tree = df.filter(items = ['Category', 'PdDistrict'])

In [27]:
# Encode categorical variables to digits
le = LabelEncoder()
X = le.fit_transform(df_tree['Category'])  
Y = le.fit_transform(df_tree['PdDistrict'])

In [30]:
# Split training and testing data 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=0)

In [42]:
#Train the decision tree classifier
clf = tree.DecisionTreeClassifier()
#The scikit-learn fit function needs a list of lists as input even if the nested lists have length 1
clf = clf.fit([X_train], [Y_train])

>* *What is the fraction of correct predictions*

In [57]:

pred = clf.predict([[5,1,2,3,4,5]])

ValueError: Number of features of the model must match the input. Model n_features is 1821478 and input n_features is 6 